In [62]:
#!npx degit metaplex-foundation/python-api/metaplex metaplex


In [63]:
import requests
from metaplex import metadata
from solana.rpc.api import Client
import json
import pandas as pd


In [64]:
client = Client("http://api.mainnet-beta.solana.com")


In [65]:
def get_atts(mint_id):
    data = metadata.get_metadata(client, mint_id)["data"]
    res = requests.get(data["uri"])
    return res.json()["attributes"]


In [66]:
# read degenape.json
with open("degenape.json") as f:
    degenape = json.load(f)


In [69]:
# get the top 100 nft's from the degenape.json
top_100_nft_traits = [get_atts(nft) for nft in degenape[:100]]


In [82]:
# create a dataframe of the top 100 nft's traits
df_all = pd.DataFrame()
for nft in top_100_nft_traits:
    df_1 = pd.DataFrame(nft)
    # add a column for the nft id
    index = top_100_nft_traits.index(nft)
    df_1["nft_id"] = degenape[index]
    # pivot the dataframe on the trait name
    df_1 = df_1.pivot(index="nft_id", columns="trait_type", values="value")

    df_all = df_all.append(df_1)


In [128]:
assets_traits = df_all.copy()


In [129]:
traits = df_all.columns.tolist()


In [130]:
for trait in traits:
    i_proportion = df_all[trait].value_counts(dropna=False, normalize=True)
    
    assets_traits = assets_traits.merge(
            i_proportion,
            left_on=trait,
            right_index=True,
            suffixes=("", "_rarity"),
            how="left",
        )


In [134]:
assets_traits.columns

Index(['Background', 'Cloths', 'EyesWear', 'HeadWear', 'Mouth', 'Skin',
       'Background_rarity', 'Cloths_rarity', 'EyesWear_rarity',
       'HeadWear_rarity', 'Mouth_rarity', 'Skin_rarity'],
      dtype='object')

In [135]:
rarity_list = assets_traits.columns.tolist()
#only include the rarity columns '_rarity'
rarity_list = [col for col in rarity_list if col.endswith("_rarity")]

In [136]:
rarity_list

['Background_rarity',
 'Cloths_rarity',
 'EyesWear_rarity',
 'HeadWear_rarity',
 'Mouth_rarity',
 'Skin_rarity']

In [137]:
assets_traits[rarity_list]

,Background_rarity,Cloths_rarity,EyesWear_rarity,HeadWear_rarity,Mouth_rarity,Skin_rarity
nft_id,,,,,,
2caeyTFigCYWtRVbYqenEJxUasBgwE7AFA7i5pgruew8,0.19,0.06,0.08,0.07,0.09,0.14
34pPEH6QMY6fxCLtmCXdVVGk6GeZwfgfE3VutqqeYnTZ,0.19,0.06,0.10,0.04,0.10,0.07
DuKd3FyLPVjmZJ1esowYCoVj2mgVVDFdWudkX1mgMZJ7,0.13,0.06,0.05,0.11,0.09,0.06
245ZP737DDFiU3FhDwnsHndv7s1vLyHsfDpdSuHsVQg2,0.16,0.06,0.06,0.03,0.14,0.14
CxaNXWrY6LMEtMk7b4iTtqUZmLMt8mPLXK6acp14oq5Q,0.13,0.12,0.09,0.08,0.19,0.06
...,...,...,...,...,...,...
D6N2QjDbp1f6wiFXY8jFmQBFtcWJrycgZrVKD94LMeWe,0.24,0.06,0.05,0.04,0.11,0.06
GmPUmEP6ENaPNn6FrazF2jL8tMZd8QGJ7MdsDwnbjkBd,0.13,0.04,0.08,0.07,0.09,0.04
HxoB2y45afTeEWEsqAXqqm2vgRSmaDLuhLdn37ddWs6j,0.24,0.06,0.10,0.11,0.15,0.05


In [124]:
rarity_list

['Background_rarity',
 'Cloths_rarity',
 'EyesWear_rarity',
 'HeadWear_rarity',
 'Mouth_rarity',
 'Skin_rarity',
 'Background_rarity_rarity',
 'Cloths_rarity_rarity',
 'EyesWear_rarity_rarity',
 'HeadWear_rarity_rarity',
 'Mouth_rarity_rarity',
 'Skin_rarity_rarity',
 'Background_rarity_rarity',
 'Cloths_rarity_rarity',
 'EyesWear_rarity_rarity',
 'HeadWear_rarity_rarity',
 'Mouth_rarity_rarity',
 'Skin_rarity_rarity']